In [15]:
import dpkt
import struct
import glob
import pandas as pd
from datetime import datetime
import itertools

# iNETx structure:
# • iNET control field (32 bits): 4 bytes
# • Stream ID (32 bits): 4 bytes
# • Sequence Number (32 bits): 4 bytes
# • Packet length Field (32 bits): 4 bytes
# • PTP Timestamp (64 bits): 8 bytes
# • iNET-X Payload Information field (32 bits): 4 bytes
# •   Error/Event Code (16 bits): 2 bytes
# •   Parameter (16 bits) 2 bytes - This is the value of the mode to which the KAD/BCU/140/F/SEF module should be switched


def ptp_timestamp_calc(inetx):
    ptp_timestamp_seconds_hex = int(inetx[16:20].hex().upper(), 16)
    ptp_timestamp_nanoseconds_hex = int(inetx[20:24].hex().upper(), 16)
    ptp_timestamp_seconds = ptp_timestamp_seconds_hex + (ptp_timestamp_nanoseconds_hex/1000000000)
    ptp_timestamp = datetime.utcfromtimestamp(ptp_timestamp_seconds).strftime('%H:%M:%S.%f')
    return ptp_timestamp

def bytes2binstr(b, n=None):
    s = ' '.join(f'{x:08b}' for x in b)
    return s if n is None else s[:n + n // 8 + (0 if n % 8 else -1)]

def get_payload_bytes(payload_content):
    for payload_bytes in payload_content:
        return [payload_content[0::, 2]]


src_ip_addresses_to_check = ['192.168.28.12']

pcap_file_list = glob.glob('_ethernet_*')
pcap_count = 1

print(f'\nAnalysing iNET-X packets coming from IP address: {src_ip_addresses_to_check[0]}')

streamid_list, timestamp_list, sequence_number_list, packet_length_list, \
error_bit_list, payload_bytes_list = ([] for _ in range(6))

for pcap_file in pcap_file_list:
    print('\nAnalysing {}. File {} or {}'.format(pcap_file, pcap_count, len(pcap_file_list)))
    print('-'*50)
    pcap_count += 1

    # --------------------------------------------------------------------------
    print('Extracting the inetx content, please wait. \n')

    ethernet_data = dpkt.pcap.Reader(open(pcap_file, 'rb'))
    for eth_ts, buf in ethernet_data:

        src_ip_raw = buf[26:30]
        src_ip = '{}.{}.{}.{}'.format(src_ip_raw[0], src_ip_raw[1], src_ip_raw[2], src_ip_raw[3])
        if src_ip in src_ip_addresses_to_check:
                #start_pc_time = datetime.utcfromtimestamp(int(eth_ts)).strftime('%Y-%m-%d %H:%M:%S')

            inetx = buf[42:]
            if inetx[:4] == b'\x11\x00\x00\x00': # The control field is correct
                streamid = inetx[4:8].hex().upper()
                sequence_number = int(inetx[8:12].hex().upper(), 16)
                packet_length = int(inetx[12:16].hex().upper(), 16)
                ptp_timestamp = ptp_timestamp_calc(inetx)
                payload_info_field = bytes2binstr(inetx[24:28])
                error_bit = int(payload_info_field[0])
                lost_count = payload_info_field[1:5]
                timeout = payload_info_field[5]
                payload = (inetx[28::].hex().upper())
                payload_bytes = [payload[i:i+4] for i in range(0, len(payload), 4) ]
                columns=['bytes_' + str(i) + '_' + str(i+1) for i in range(1, 2*len(payload_bytes), 2)]
                streamid_list.append(streamid)
                timestamp_list.append(ptp_timestamp)
                sequence_number_list.append(sequence_number)
                packet_length_list.append(packet_length)
                error_bit_list.append(error_bit)
                payload_bytes_list.append(payload_bytes)


df = pd.DataFrame([streamid_list, sequence_number_list, packet_length_list, timestamp_list, error_bit_list, payload_bytes_list],
                  index = ['STREAM_ID', 'sequence_number', 'packet_length', 'timestamp', 'error_bit', 'payload_bytes']).T
df

                #df = pd.DataFrame([payload_bytes], columns = columns)


                #print(payload, '\n\n\n',  payload_bytes)


                #payload = itertools.islice(inetx[28::].hex(), 0 , None, 2)
                #print(streamid, '\t\t', sequence_number,'\t\t', packet_length, '\t\t', payload)





Analysing iNET-X packets coming from IP address: 192.168.28.12

Analysing _ethernet_.pcap25. File 1 or 2
--------------------------------------------------
Extracting the inetx content, please wait. 


Analysing _ethernet_.pcap49. File 2 or 2
--------------------------------------------------
Extracting the inetx content, please wait. 



,STREAM_ID,sequence_number,packet_length,timestamp,error_bit,payload_bytes
0,0000FEEB,167691,284,02:21:09.406250,0,"[2572, 5C35, EF9A, 9879, FBAA, 0BFD, 7FBB, EFC..."
1,02FFFFFF,1035806,386,02:21:09.403000,0,"[80B3, A70C, A000, A700, A701, A702, A703, A70..."
2,0000FEEA,127530,36,02:21:09.412500,0,"[8069, 806E, 806C, 8071]"
3,0000FEE9,86614,44,02:21:09.412500,0,"[806E, 8070, 806F, 8071, 806E, 806F, 806C, 806E]"
4,0000FEE8,1355010,44,02:21:09.412500,0,"[8070, 806C, 806E, 806D, 806D, 806E, 806A, 806E]"
...,...,...,...,...,...,...
235289,0000FF2C,791975,44,03:34:37.918750,0,"[806D, 806D, 806E, 8070, 8068, 806F, 806B, 8067]"
235290,0000FF2B,2060371,44,03:34:37.918750,0,"[806B, 806A, 806B, 806B, 806A, 806B, 8069, 806C]"
235291,0000FF2E,832891,284,03:34:37.918750,0,"[ACA0, 07A6, 2AEB, 6A8F, 6604, A6E1, 5BE3, 000..."
235292,0000FF2D,832892,36,03:34:37.925000,0,"[806E, 806E, 806F, 806E]"


In [27]:
len(df['payload_bytes'][2])

4

In [29]:
df[['1','2', '3', '4']] = df['payload_bytes'].str.split(',', 3, expand=True)
df

ValueError: Columns must be same length as key

In [23]:
len(df['payload_bytes'].min())

128